In [29]:
import requests
import json

API = 'http://localhost:5000'
session = requests.Session()

# Building your own language in python

<p style='text-align: right;'> David De Sousa - WeRiot </p>

# What is a Domain Specific Language (DSL)

# Advantages and Disadvantages

# Why use a DSL 

# Real world DSLs you (may) use every day

* HTML
* CSS
* LaTeX
* SQL
* Jinja

# Coming next

* Implementing a math endpoint "the regular way"
* Implementing with a DSL
* WeRiot use case
* (maybe) Extending the DSL live!
* Questions

# Sample use case

## Client requests a "sum" endpoint


```python
@app.route('/sum')
def sum():
    # skip validations because reasons
    a = float(request.args.get('param1'))
    b = float(request.args.get('param2'))

    return jsonify(result=a + b)
```

done!

In [30]:
params = (5, 4)

resp = session.get(
    f'{API}/sum', 
    params={'param1': params[0], 'param2': params[1]},
)
print(resp.text)

{
  "result": 9.0
}



# Sample use case

#### "Super awesome, but now I want to sum 3 elements"

* Add support in endpoint
* Tell client to call API twice

```python
@app.route('/sum2')
def sum2():
    # skip validations because reasons
    a = float(request.args.get('param1'))
    b = float(request.args.get('param2'))
    c = 0

    if 'param3' in request.args:
        c = float(request.args.get('param3'))

    return jsonify(result=a + b + c)
```

In [31]:
params = (5, 4, 7)
payload = {'param1': params[0], 'param2': params[1]}
if len(params) > 2:
    payload['param3'] = params[2]

resp = session.get(
    f'{API}/sum2', 
    params=payload,
)
print(resp.text)

{
  "result": 16.0
}



# Sample use case

#### Client wants substraction

#### And division
#### And multiplication
#### All at the same time

# Sample use case

## Let's use json!

34 + (2 * 3)
```json
{
"operator": "+",
"param1": 34,
"param2": {
    "operator": "*",
    "param1": 2,
    "param2": 3 }
}
```

```python
def process_operation(operation):
    operators = {
        '*': operator.mul,
        '/': operator.truediv,
        '+': operator.add,
        '-': operator.sub
    }

    param1 = operation['param1']
    param2 = operation['param2']
    operator_function = operators[operation['operator']]

    if type(param1) is dict:
        param1 = process_operation(param1)

    if type(param2) is dict:
        param2 = process_operation(param2)

    return operator_function(param1, param2)


@app.route('/math')
def math():
    operation = json.loads(request.args.get('operation'))

    return jsonify(result=process_operation(operation))
```

In [32]:
operation = {  #  (6/3) * (7-2)
    "operator": "*", 
    "param1": {
        "operator": "/", 
        "param1": 6, 
        "param2": 3
     }, 
     "param2": {
         "operator": "-",
         "param1": 7,
         "param2": 2
     }
}

payload = {"operation": json.dumps(operation)}
resp = session.get(
    f'{API}/math', 
    params=payload,
)
print(resp.text)

{
  "result": 10.0
}



# Sample use case

* Solved!
* Flexible API
* One call can do all the calculations
* Simple, right?

# Sample use case

`((2 + (3 * 4) - 5) * 6) / 4`

```json
{
    "operator": "/",
    "param1": {
        "operator": "*",
        "param1": {
            "operator": "+",
            "param1": 2,
            "param2": {
                "operator": "-",
                "param1": {
                    "operator": "*",
                    "param1": 3,
                    "param2": 4
                },
                "param2": 5
            }
        },
        "param2": 6
    },
    "param2": 4
}
```

# Client is now crying

#### Could we do better?

Which format would be easier for the user?

How about:

`((2 + (3 * 4) - 5) * 6) / 4`

# Your first DSL

## Lexer

Split every component of the language in tokens using regular expressions.

```python
from sly import Lexer, Parser
from werkzeug import exceptions


class CalcLexer(Lexer):
    tokens = {NUMBER}
    ignore = ' \t\n'
    literals = {'+'}

    @_(r'\d+(\.\d+)?')
    def NUMBER(self, t):
        t.value = float(t.value)
        return t

    def error(self, t):
        raise exceptions.BadRequest(f"Illegal sequence '{t.value}'")
```

## Parser

Use the tokens to define your language grammar

```python
class CalcParser(Parser):
    tokens = CalcLexer.tokens

    def error(self, p):
        if p is None:
            raise exceptions.BadRequest("Syntax error, unexpected EOF")
        raise exceptions.BadRequest(
            f"Syntax error near '{p.value}', line={p.lineno} column={p.index}")

    @_('operation')
    def statement(self, p):
        return p.operation
    
    # operation : NUMBER
    #           | NUMBER + NUMBER

    @_('NUMBER')
    def operation(self, p):
        return p.NUMBER

    @_('NUMBER "+" NUMBER')
    def operation(self, p):
        return p.NUMBER0 + p.NUMBER1
```

# Your first DSL

## Sum with infinite params

```python
class CalcParser(Parser):
    tokens = CalcLexer.tokens

    def error(self, p):
        if p is None:
            raise exceptions.BadRequest("Syntax error, unexpected EOF")
        raise exceptions.BadRequest(
            f"Syntax error near '{p.value}', line={p.lineno} column={p.index}")

    @_('operation')
    def statement(self, p):
        return p.operation
    
    # operation : NUMBER                  (1)
    #           | operation + operation   (2)

    @_('NUMBER')
    def operation(self, p):
        return p.NUMBER

    @_('operation "+" operation')
    def operation(self, p):
        return p.operation0 + p.operation1
```

`WARNING: 1 shift/reduce conflicts`

```
operation : NUMBER                  (1)
          | operation + operation   (2)
```

<p><img alt="Conflict explanation" src="images/conflict.svg" class="conflict-image" /></p>

## Conflict Resolution

```python
class CalcParser(Parser):
    tokens = CalcLexer.tokens

    precedence = (
        ('left', '+'),
    )

    def error(self, p):
        if p is None:
            raise exceptions.BadRequest("Syntax error, unexpected EOF")
        raise exceptions.BadRequest(
            f"Syntax error near '{p.value}', line={p.lineno} column={p.index}")

    @_('operation')
    def statement(self, p):
        return p.operation

    @_('NUMBER')
    def operation(self, p):
        return p.NUMBER

    @_('operation "+" operation')
    def operation(self, p):
        return p.operation0 + p.operation1
```

In [33]:
payload = {"operation": '3 + 4 + 5 + 6'}
resp = session.get(
    f'{API}/math2', 
    params=payload,
)
print(resp.text)

{
  "result": 18.0
}



# Your first DSL

### Complete Lexer

```python
class CalcLexer(Lexer):
    tokens = {NUMBER}
    ignore = ' \t\n'
    literals = {'+', '-', '/', '*', '(', ')'}

    @_(r'\d+(\.\d+)?')
    def NUMBER(self, t):
        t.value = float(t.value)
        return t

    def error(self, t):
        raise exceptions.BadRequest(f"Illegal sequence '{t.value}'")
```

### Complete parser

```python
class CalcParser(Parser):
    tokens = CalcLexer.tokens

    precedence = (
        ('left', '+', '-'),
        ('left', '*', '/'),
    )

    def error(self, p):
        if p is None:
            raise exceptions.BadRequest("Syntax error, unexpected EOF")
        raise exceptions.BadRequest(
            f"Syntax error near '{p.value}', line={p.lineno} column={p.index}")

    @_('operation')
    def statement(self, p):
        return p.operation
    
    # operation : NUMBER
    #           | ( operation )
    #           | operation + operation
    #           | operation - operation
    #           | operation / operation
    #           | operation * operation

    @_('NUMBER')
    def operation(self, p):
        return p.NUMBER

    @_('"(" operation ")"')
    def operation(self, p):
        return p.operation

    @_('operation "+" operation')
    def operation(self, p):
        return p.operation0 + p.operation1

    @_('operation "*" operation')
    def operation(self, p):
        return p.operation0 * p.operation1

    @_('operation "/" operation')
    def operation(self, p):
        return p.operation0 / p.operation1

    @_('operation "-" operation')
    def operation(self, p):
        return p.operation0 - p.operation1
```

### Same language, different implementation

```python
class CalcLexer(Lexer):
    tokens = {NUMBER, PLUS, MINUS, DIVIDE, TIMES}
    ignore = ' \t\n'
    literals = {'(', ')'}

    @_(r'\d+(\.\d+)?')
    def NUMBER(self, t):
        t.value = float(t.value)
        return t

    @_(r'\+')
    def PLUS(self, t):
        t.value = operator.add
        return t

    @_(r'-')
    def MINUS(self, t):
        t.value = operator.sub
        return t

    @_(r'/')
    def DIVIDE(self, t):
        t.value = operator.truediv
        return t

    @_(r'\*')
    def TIMES(self, t):
        t.value = operator.mul
        return t

    def error(self, t):
        raise exceptions.BadRequest(f"Illegal sequence '{t.value}'")
```

```python
class CalcParser(Parser):
    tokens = CalcLexer.tokens

    precedence = (
        ('left', PLUS, MINUS),
        ('left', DIVIDE, TIMES),
    )

    def error(self, p):
        if p is None:
            raise exceptions.BadRequest("Syntax error, unexpected EOF")
        raise exceptions.BadRequest(
            f"Syntax error near '{p.value}', line={p.lineno} column={p.index}")

    @_('operation')
    def statement(self, p):
        return p.operation
    
    # operation : NUMBER
    #           | ( operation )
    #           | operation PLUS operation
    #           | operation MINUS operation
    #           | operation TIMES operation
    #           | operation DIVIDE operation

    @_('NUMBER')
    def operation(self, p):
        return p.NUMBER

    @_('"(" operation ")"')
    def operation(self, p):
        return p.operation

    @_(
        'operation PLUS operation',
        'operation MINUS operation',
        'operation TIMES operation',
        'operation DIVIDE operation',
    )
    def operation(self, p):
        return p[1](p.operation0, p.operation1)
```

# Client is now crying (of happiness)

In [34]:
payload = {"operation": '((2 + (3 * 4) - 4) * 2) / 4'}
resp = session.get(
    f'{API}/math2', 
    params=payload,
)
print(resp.text)

{
  "result": 5.0
}



# WeRiot Cella Query Language

### Cella

* Generic REST storage for JSON objects
* Filtering
* Aggregations
* JOINs based on object contents
* Time series queries support

```bash
curl http://cella/events/sample1 -d '{"data": {"temperature": 45, "sensor": "A"}}'
curl http://cella/events/sample1 -d '{"data": {"temperature": 60, "sensor": "B"}}'
curl http://cella/events/sample1 -d '{"data": {"temperature": 40, "sensor": "A"}}'
curl http://cella/events/sample1 -d '{"data": {"temperature": 40, "sensor": "A"}}'
```



#### Querying
`http://cella/objects/query?query='<WCQL>'`

##### filter based on values:
```
from("sample1") where(data.sensor == 'A')

from("sample1") where(data.sensor in ['A', 'B'] and data.temperature > 40)
```

##### aggregations:
```
from("sample1") aggregate(avg(data.temperature)) group(data.sensor)

from("sample1") aggregate(avg(data.temperature)) group(data.sensor) timebucket('00:10:00')
```


##### multiple atomic inserts
```
insert("profiles") '{"id": "test1", "data": {"name": "john"}}';
insert("profiles") '{"id": "test2", "data": {"name": "joe"}}';
insert("profiles") '{"id": "test3", "data": {"name": "david"}}';
insert("groups") '{"id": "agroup", "data": {"name": "mygroup"}}';
insert("group_members") '{"data": {"profile": "test1", "group": "agroup"}}';
insert("group_members") '{"data": {"profile": "test2", "group": "agroup"}}';
insert("group_members") '{"data": {"profile": "test3", "group": "agroup"}}'
```

##### JOINs:
```
from("profiles") where(id in (
    from("group_members") select(data.profile) where(data.name == 'mygroup')
    )
)
```

# Add cosine function to the DSL

# Where to go from here

* slides and sample code https://github.com/dedsm/dsl_python
* SLY author giving a better talk than this one https://youtu.be/zJ9z6Ge-vXs
* SLY homepage https://sly.readthedocs.io/en/latest/
* Book with all the theory https://en.wikipedia.org/wiki/Compilers:_Principles,_Techniques,_and_Tools

# Thanks